In [27]:
# tensorflow version
import tensorflow as tf
# check tensorflow version
print(tf.__version__)

2.17.0


In [28]:
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)




In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()


/opt/anaconda3/envs/ml-zoomcamp39/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

Prepare Data Generators

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'hairdataset/data/train', 
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    'hairdataset/data/test', 
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)


Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


Compile the model

In [31]:
from tensorflow.keras.optimizers import SGD

model.compile(
    optimizer=SGD(learning_rate=0.002, momentum=0.8),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


Train the model

In [32]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.5047 - loss: 0.7181 - val_accuracy: 0.6219 - val_loss: 0.6816
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 452ms/step - accuracy: 0.6646 - loss: 0.6676 - val_accuracy: 0.6070 - val_loss: 0.6626
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 459ms/step - accuracy: 0.6617 - loss: 0.6268 - val_accuracy: 0.6318 - val_loss: 0.6285
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 165ms/step - accuracy: 0.6818 - loss: 0.5942 - val_accuracy: 0.5622 - val_loss: 0.7995
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 149ms/step - accuracy: 0.6814 - loss: 0.6179 - val_accuracy: 0.6368 - val_loss: 0.6100
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 155ms/step - accuracy: 0.7350 - loss: 0.5324 - val_accuracy: 0.6468 - val_loss: 0.6191
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 154ms/step - accuracy: 0.7400 - loss: 0.4974 - val_accuracy: 0.6716 - val_loss: 0.5974
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - accuracy: 0.7428 - loss: 0.5104 - val_accuracy: 0.

Median Training Accuracy

In [33]:
import numpy as np

training_accuracies = history.history['accuracy']
median_accuracy = np.median(training_accuracies)
print(median_accuracy)


0.7118749916553497


Standard Deviation of Training Loss


In [34]:
training_losses = history.history['loss']
std_loss = np.std(training_losses)
print(std_loss)


0.07236436970444173


Adding data augmentations:

In [35]:
augmented_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

augmented_train_generator = augmented_train_datagen.flow_from_directory(
    'hairdataset/data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)


Found 800 images belonging to 2 classes.


In [36]:

from tensorflow.keras.optimizers import SGD

model.compile(
    optimizer=SGD(learning_rate=0.002, momentum=0.8),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


continue traininf

In [37]:
history_augmented = model.fit(
    augmented_train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 223ms/step - accuracy: 0.6457 - loss: 0.6303 - val_accuracy: 0.6169 - val_loss: 0.6731
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 218ms/step - accuracy: 0.6689 - loss: 0.6287 - val_accuracy: 0.7015 - val_loss: 0.5753
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 223ms/step - accuracy: 0.6418 - loss: 0.6291 - val_accuracy: 0.6816 - val_loss: 0.6097
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 222ms/step - accuracy: 0.6938 - loss: 0.5915 - val_accuracy: 0.7214 - val_loss: 0.5621
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 236ms/step - accuracy: 0.6349 - loss: 0.6351 - val_accuracy: 0.7164 - val_loss: 0.5585
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 225ms/step - accuracy: 0.7311 - loss: 0.5722 - val_accuracy: 0.7114 - val_loss: 0.5651
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 226ms/step - accuracy: 0.6960 - loss: 0.5712 - val_accuracy: 0.6965 - val_loss: 0.5581
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 225ms/step - accuracy: 0.7260 - loss: 0.5579 - val_accuracy: 0.

Calculate mean test loss:

In [38]:
test_losses = history_augmented.history['val_loss']
mean_test_loss = np.mean(test_losses)
print(mean_test_loss)

#


0.5797339677810669


Calculate average test accuracy for epochs 6–10

In [39]:
test_accuracies = history_augmented.history['val_accuracy'][5:]
average_test_accuracy = np.mean(test_accuracies)
print(average_test_accuracy)


0.7094527363777161
